# arXiv Research Intelligence Analysis

This notebook provides interactive analysis of the collected arXiv dataset.

**What you can explore:**
- Research trends over time
- Citation patterns and success factors
- Research gaps and opportunities
- Author networks and influence
- Method and topic evolution

In [ ]:
# Setup and imports
import json
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
from datetime import datetime

# Visualization
try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    sns.set_style('whitegrid')
    plt.rcParams['figure.figsize'] = (12, 6)
    HAS_PLOTTING = True
except ImportError:
    print("Install matplotlib and seaborn for visualizations:")
    print("  uv add matplotlib seaborn")
    HAS_PLOTTING = False

print("Setup complete!")

In [ ]:
# Load data
DATA_DIR = "../arxiv_research_data"  # Adjust path as needed

# Load papers
with open(f"{DATA_DIR}/papers.json", "r") as f:
    papers = json.load(f)

df = pd.DataFrame(papers)
print(f"Loaded {len(df):,} papers")
print(f"Date range: {df['published_date'].min()} to {df['published_date'].max()}")
df.head()

In [ ]:
# Load analysis results
ANALYSIS_DIR = f"{DATA_DIR}/analysis"

def load_analysis(name):
    try:
        with open(f"{ANALYSIS_DIR}/{name}.json", "r") as f:
            return json.load(f)
    except FileNotFoundError:
        return None

text_analysis = load_analysis("text_analysis")
trend_analysis = load_analysis("trend_analysis")
gap_analysis = load_analysis("gap_analysis")
impact_analysis = load_analysis("impact_analysis")
network_analysis = load_analysis("network_analysis")

print("Analysis files loaded!")

## 1. Dataset Overview

In [ ]:
# Basic statistics
print("=" * 50)
print("DATASET STATISTICS")
print("=" * 50)

print(f"\nTotal papers: {len(df):,}")
print(f"Unique categories: {df['primary_category'].nunique()}")
print(f"Unique authors: {len(set(a for authors in df['authors'].dropna() for a in authors.split('; ')))}")

# Citation stats
if 'citations' in df.columns:
    citations = df['citations'].dropna()
    print(f"\nCitation Statistics:")
    print(f"  Total: {citations.sum():,.0f}")
    print(f"  Mean: {citations.mean():.1f}")
    print(f"  Median: {citations.median():.0f}")
    print(f"  Max: {citations.max():.0f}")

# Coverage
print(f"\nData Coverage:")
print(f"  Has code: {df['has_code'].sum() if 'has_code' in df.columns else 'N/A'} ({100*df['has_code'].mean():.1f}%)" if 'has_code' in df.columns else "")
print(f"  Has TL;DR: {df['tldr'].notna().sum()} ({100*df['tldr'].notna().mean():.1f}%)" if 'tldr' in df.columns else "")
print(f"  Has venue: {df['publication_venue'].notna().sum()} ({100*df['publication_venue'].notna().mean():.1f}%)" if 'publication_venue' in df.columns else "")

In [ ]:
# Papers by category
if HAS_PLOTTING:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Top 15 categories by paper count
    cat_counts = df['primary_category'].value_counts().head(15)
    ax1 = axes[0]
    cat_counts.plot(kind='barh', ax=ax1, color='steelblue')
    ax1.set_xlabel('Number of Papers')
    ax1.set_ylabel('Category')
    ax1.set_title('Top 15 Categories by Paper Count')
    ax1.invert_yaxis()
    
    # Papers over time
    ax2 = axes[1]
    df['month'] = df['published_date'].str[:7]
    monthly = df.groupby('month').size()
    monthly.plot(ax=ax2, marker='o', color='steelblue')
    ax2.set_xlabel('Month')
    ax2.set_ylabel('Papers')
    ax2.set_title('Papers Published Over Time')
    ax2.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()

## 2. Research Trends

In [ ]:
# Emerging terms
if text_analysis and 'emerging_terms' in text_analysis:
    emerging = text_analysis['emerging_terms'].get('emerging_terms', {})
    
    print("EMERGING TERMS (growing in frequency)")
    print("=" * 50)
    
    emerging_df = pd.DataFrame([
        {'term': k, **v} for k, v in list(emerging.items())[:30]
    ])
    
    if len(emerging_df) > 0:
        display(emerging_df)

In [ ]:
# Method trends
if trend_analysis and 'method_trends' in trend_analysis:
    methods = trend_analysis['method_trends']
    
    print("EMERGING METHODS")
    print("=" * 50)
    
    emerging_methods = methods.get('emerging_methods', [])
    if emerging_methods:
        em_df = pd.DataFrame(emerging_methods)
        display(em_df)
    
    print("\nDECLINING METHODS")
    print("=" * 50)
    
    declining_methods = methods.get('declining_methods', [])
    if declining_methods:
        dm_df = pd.DataFrame(declining_methods)
        display(dm_df)

In [ ]:
# Category trends
if trend_analysis and 'category_trends' in trend_analysis:
    cat_trends = trend_analysis['category_trends']
    
    trends_df = pd.DataFrame([
        {'category': k, **{kk: vv for kk, vv in v.items() if kk != 'time_series'}}
        for k, v in cat_trends.items()
    ]).sort_values('momentum', ascending=False)
    
    print("CATEGORY TRENDS BY MOMENTUM")
    print("=" * 50)
    display(trends_df.head(20))

In [ ]:
# Hot papers (high citation velocity)
if trend_analysis and 'hot_papers' in trend_analysis:
    hot = trend_analysis['hot_papers']
    
    print("🔥 HOT PAPERS (Highest Citation Velocity)")
    print("=" * 50)
    
    for i, paper in enumerate(hot[:15], 1):
        print(f"\n{i}. {paper['title'][:80]}{'...' if len(paper['title']) > 80 else ''}")
        print(f"   Citations: {paper['citations']} | Velocity: {paper['citations_per_month']:.2f}/month")
        print(f"   Category: {paper['category']} | Has Code: {paper.get('has_code', False)}")

## 3. Research Opportunities

In [ ]:
# Top opportunities
if gap_analysis and 'opportunity_scores' in gap_analysis:
    opportunities = gap_analysis['opportunity_scores'].get('ranked_opportunities', [])
    
    print("🎯 TOP RESEARCH OPPORTUNITIES")
    print("=" * 50)
    
    opp_df = pd.DataFrame(opportunities[:20])
    if len(opp_df) > 0:
        display(opp_df[['category', 'opportunity_score', 'paper_count', 'avg_citations', 
                        'avg_velocity', 'growth_indicator', 'recommendation']])

In [ ]:
# Underexplored intersections
if gap_analysis and 'category_intersections' in gap_analysis:
    intersections = gap_analysis['category_intersections'].get('underexplored_pairs', [])
    
    print("🔍 UNDEREXPLORED CATEGORY INTERSECTIONS")
    print("=" * 50)
    print("These category combinations have fewer papers than expected:")
    print()
    
    for inter in intersections[:15]:
        cats = ' + '.join(inter['categories'])
        print(f"  {cats}")
        print(f"    Actual: {inter['actual_papers']} | Expected: {inter['expected_papers']} | Opportunity: {inter['opportunity_score']}")

In [ ]:
# Survey opportunities
if gap_analysis and 'survey_opportunities' in gap_analysis:
    surveys = gap_analysis['survey_opportunities'].get('category_survey_opportunities', [])
    
    print("📚 SURVEY PAPER OPPORTUNITIES")
    print("=" * 50)
    print("Categories with many papers but few/no surveys:")
    print()
    
    survey_df = pd.DataFrame(surveys[:15])
    if len(survey_df) > 0:
        display(survey_df)

In [ ]:
# Reproducibility gaps
if gap_analysis and 'reproducibility_gaps' in gap_analysis:
    repro = gap_analysis['reproducibility_gaps']
    
    print("🔧 REPRODUCIBILITY OPPORTUNITIES")
    print("=" * 50)
    print("Highly-cited papers without code (implement for impact):")
    print()
    
    no_code = repro.get('high_cited_no_code', [])
    for paper in no_code[:10]:
        print(f"  [{paper['citations']} citations] {paper['title'][:60]}...")
        print(f"    Category: {paper['category']} | Methods: {', '.join(paper.get('methods', [])[:3])}")

## 4. Success Factors

In [ ]:
# What makes papers successful?
if impact_analysis:
    print("🏆 WHAT MAKES PAPERS SUCCESSFUL")
    print("=" * 50)
    
    # Feature analysis
    features = impact_analysis.get('feature_analysis', {})
    
    print("\nBinary Features Impact:")
    for name, data in features.items():
        if 'citation_lift' in data:
            impact = data['impact']
            lift = data['citation_lift']
            symbol = '✅' if impact == 'positive' else '❌' if impact == 'negative' else '➖'
            print(f"  {symbol} {name}: {lift:.2f}x citations (with: {data['avg_citations_with']:.1f}, without: {data['avg_citations_without']:.1f})")
    
    # Success formula
    formula = impact_analysis.get('success_formula', {})
    if formula:
        print(f"\n📊 Success Formula:")
        print(f"   {formula.get('interpretation', '')}")
        print(f"\n   Top factors: {', '.join(formula.get('top_factors', []))}")

In [ ]:
# Citation distribution analysis
if HAS_PLOTTING and 'citations' in df.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Citation distribution (log scale)
    ax1 = axes[0]
    citations = df['citations'].dropna()
    citations_nonzero = citations[citations > 0]
    ax1.hist(np.log10(citations_nonzero + 1), bins=50, color='steelblue', edgecolor='white')
    ax1.set_xlabel('log10(Citations + 1)')
    ax1.set_ylabel('Number of Papers')
    ax1.set_title('Citation Distribution')
    
    # Citations by category
    ax2 = axes[1]
    top_cats = df['primary_category'].value_counts().head(10).index
    cat_citations = df[df['primary_category'].isin(top_cats)].groupby('primary_category')['citations'].mean().sort_values(ascending=True)
    cat_citations.plot(kind='barh', ax=ax2, color='steelblue')
    ax2.set_xlabel('Average Citations')
    ax2.set_ylabel('Category')
    ax2.set_title('Average Citations by Category')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Code availability impact
if 'has_code' in df.columns and 'citations' in df.columns:
    with_code = df[df['has_code'] == True]['citations'].dropna()
    without_code = df[df['has_code'] == False]['citations'].dropna()
    
    print("CODE AVAILABILITY IMPACT")
    print("=" * 50)
    print(f"Papers WITH code:    {len(with_code):,} papers, avg citations: {with_code.mean():.1f}")
    print(f"Papers WITHOUT code: {len(without_code):,} papers, avg citations: {without_code.mean():.1f}")
    print(f"\nCitation multiplier: {with_code.mean() / without_code.mean():.2f}x")

## 5. Author Analysis

In [ ]:
# Top authors
if network_analysis and 'author_metrics' in network_analysis:
    author_metrics = network_analysis['author_metrics']
    
    print("👥 MOST INFLUENTIAL AUTHORS")
    print("=" * 50)
    
    # By citations
    print("\nTop by Total Citations:")
    for a in author_metrics.get('top_productive', [])[:10]:
        print(f"  {a['name'][:40]}: {a['paper_count']} papers")

In [ ]:
# Team size analysis
if 'author_count' in df.columns:
    print("TEAM SIZE ANALYSIS")
    print("=" * 50)
    
    team_stats = df.groupby(pd.cut(df['author_count'], bins=[0, 1, 3, 6, 100], 
                                    labels=['Solo', 'Small (2-3)', 'Medium (4-6)', 'Large (7+)'])).agg({
        'arxiv_id': 'count',
        'citations': 'mean'
    }).rename(columns={'arxiv_id': 'papers', 'citations': 'avg_citations'})
    
    display(team_stats)

## 6. Method & Dataset Analysis

In [ ]:
# Most common methods
if text_analysis and 'method_frequency' in text_analysis:
    methods = text_analysis['method_frequency']
    
    print("🔧 MOST USED METHODS")
    print("=" * 50)
    
    method_df = pd.DataFrame(list(methods.items()), columns=['method', 'count']).head(25)
    display(method_df)

In [ ]:
# Most mentioned datasets
if text_analysis and 'dataset_frequency' in text_analysis:
    datasets = text_analysis['dataset_frequency']
    
    print("📊 MOST USED DATASETS")
    print("=" * 50)
    
    dataset_df = pd.DataFrame(list(datasets.items()), columns=['dataset', 'count']).head(25)
    display(dataset_df)

## 7. Custom Analysis

Use this section for your own explorations.

In [ ]:
# Example: Find papers about a specific topic
def search_papers(query, n=20):
    """Search papers by title/abstract."""
    query = query.lower()
    matches = df[
        df['title'].str.lower().str.contains(query, na=False) |
        df['abstract'].str.lower().str.contains(query, na=False)
    ].nlargest(n, 'citations' if 'citations' in df.columns else 'arxiv_id')
    
    return matches[['title', 'citations', 'primary_category', 'published_date', 'arxiv_url']]

# Example: search_papers("graph neural")

In [ ]:
# Example: Find highest-cited papers in a category
def top_papers_in_category(category, n=15):
    """Get top-cited papers in a category."""
    cat_papers = df[df['primary_category'] == category].nlargest(n, 'citations')
    return cat_papers[['title', 'citations', 'citations_per_month', 'has_code', 'arxiv_url']]

# Example: top_papers_in_category('cs.LG')

In [ ]:
# Example: Compare two categories
def compare_categories(cat1, cat2):
    """Compare two categories."""
    data = []
    for cat in [cat1, cat2]:
        subset = df[df['primary_category'] == cat]
        data.append({
            'category': cat,
            'papers': len(subset),
            'avg_citations': subset['citations'].mean() if 'citations' in subset else None,
            'code_rate': subset['has_code'].mean() * 100 if 'has_code' in subset else None,
            'avg_authors': subset['author_count'].mean() if 'author_count' in subset else None,
        })
    return pd.DataFrame(data)

# Example: compare_categories('cs.LG', 'cs.CV')

In [ ]:
# Your exploration here!
# Try:
#   search_papers("your topic")
#   top_papers_in_category("cs.AI")
#   compare_categories("cs.LG", "stat.ML")